In [3]:
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import pandas as pd
import time
import numpy as np
import copy
import glob
import sys, os
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import pickle
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import os
from torch.utils.data import Dataset
import numpy as np
import torch.nn.functional as F
import pandas as pd
from scipy.stats import multivariate_normal
import torch, random, copy, os

from torch.utils.data import DataLoader, Subset, random_split

In [4]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)

In [5]:
features, label, group = ACSEmployment.df_to_pandas(acs_data)

In [6]:
merged_df=pd.concat([features, label], axis=1)
merged_df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,False


In [5]:
merged_df.columns

Index(['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR'],
      dtype='object')

In [6]:
# merged_df=merged_df[['AGEP', 'MAR', 'MIG', 'MIL', 'ANC','NATIVITY', 'DREM', 'SEX', 'RAC1P', 'ESR']]

# merged_df=merged_df[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DREM','SEX','RAC1P']]

In [21]:
shuffled = merged_df.sample(frac=1)
shuffled

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
13427,75.0,16.0,1.0,0.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,False
45872,20.0,16.0,5.0,4.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,True
47565,66.0,19.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,1.0,True
11393,76.0,17.0,2.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,False
194,49.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28751,30.0,16.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,False
15842,43.0,18.0,4.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,True
2675,7.0,3.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
12961,57.0,20.0,1.0,1.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,9.0,False


In [22]:
import numpy as np

total_number = len(merged_df)
print("total_number: ",total_number)

num_parts = 4
lower_limit = 3000
upper_limit = 10000

# Generate 9 random numbers between lower_limit and upper_limit
random_numbers = np.random.randint(lower_limit, upper_limit + 1, num_parts - 1)

# Sort the random numbers to ensure variety and add the remaining to the last number
random_numbers = np.sort(random_numbers)
random_numbers = np.append(random_numbers, total_number - np.sum(random_numbers))

print("Random numbers:", random_numbers)
print("Sum:", np.sum(random_numbers))


total_number:  47777
Random numbers: [ 7351  8629  9564 22233]
Sum: 47777


In [23]:
all_df_split=np.split(merged_df.sample(frac=1, random_state=42), np.cumsum(random_numbers)[:-1])

/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [24]:
temp=0
for i in all_df_split:
    print("len of Dfs",len(i))
    temp+=len(i)
    
temp

len of Dfs 7351
len of Dfs 8629
len of Dfs 9564
len of Dfs 22233


47777

In [26]:
list_client_df=[]

output_directory="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/client_df"
encoder = LabelEncoder()

for idx, ind_df in enumerate(all_df_split):
    
    x_train=0
    y_train=0

    # ['AGEP', 'MAR', 'MIG', 'MIL', 'ANC','NATIVITY', 'DREM', 'SEX', 'RAC1P', 'ESR']
    x_train = ind_df[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'SEX', 'RAC1P']]
    X_train_sens= ind_df[['SEX']]
    y_train = ind_df[['ESR']]
    
    output_filename = os.path.join(output_directory, f"client_{idx}.csv")
    ind_df.to_csv(output_filename, index=False)
    
    x_train = preprocessing.normalize(x_train)
    X_train_sens=X_train_sens['SEX'].map({1: 1, 2: 0}).to_numpy()
    
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    
    X_train= torch.tensor(x_train, dtype=torch.float32)
    X_train_sens = torch.tensor(X_train_sens, dtype=torch.float32).reshape(-1, 1)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    
    final_data = data_utils.TensorDataset(X_train, y_train,X_train_sens)
    
    list_client_df.append(final_data)

/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/p

In [12]:
# encoder.inverse_transform()

In [27]:
valset_ratio=0.05
testset_ratio=0.25
BATCH=64

client_data_for_training=[]
client_data_for_validation=[]
client_data_for_testing=[]

for i in range(len(all_df_split)):
    val_samples_num = int(len(list_client_df[i]) * valset_ratio)
    test_samples_num = int(len(list_client_df[i]) * testset_ratio)
    train_samples_num = len(list_client_df[i]) - val_samples_num - test_samples_num


    trainset, valset, testset = random_split(
            list_client_df[i], [train_samples_num, val_samples_num, test_samples_num])

    trainloader = data_utils.DataLoader(trainset, batch_size=BATCH, shuffle=True)
    valloader = data_utils.DataLoader(valset, batch_size=BATCH)
    testloader = data_utils.DataLoader(testset, batch_size=BATCH)

    client_data_for_training.append(trainloader)
    client_data_for_validation.append(valloader)
    client_data_for_testing.append(testloader)

    print(i,":Client:",len(list_client_df[i]),":Va:",val_samples_num," :Test: ",test_samples_num," :Train: ",train_samples_num)

0 :Client: 7351 :Va: 367  :Test:  1837  :Train:  5147
1 :Client: 8629 :Va: 431  :Test:  2157  :Train:  6041
2 :Client: 9564 :Va: 478  :Test:  2391  :Train:  6695
3 :Client: 22233 :Va: 1111  :Test:  5558  :Train:  15564


In [28]:
with open("/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/data_random_split_fairFed/clients_training.pkl", "wb") as f:
    pickle.dump(client_data_for_training, f)
    
with open("/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/data_random_split_fairFed/clients_validation.pkl", "wb") as f:
    pickle.dump(client_data_for_validation, f)
    
with open("/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/data_random_split_fairFed/clients_testing.pkl", "wb") as f:
    pickle.dump(client_data_for_testing, f)